In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/5_AIADVANCE')

In [ ]:
# !unzip /content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats.zip -d /content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate, BatchNormalization, Dropout, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from imutils import paths
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Tiền xử lí dữ liệu

In [4]:
root_dir = '/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/'

# Đường dẫn tới thư mục con 'train', 'val' và 'test'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

In [ ]:
# import os
# import shutil
# import random

# # Đường dẫn tới thư mục gốc
# file_dir = '/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats'
# root_dir = '/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/'

# # Đường dẫn tới thư mục con 'train', 'val' và 'test'
# train_dir = os.path.join(root_dir, 'train')
# val_dir = os.path.join(root_dir, 'val')
# test_dir = os.path.join(root_dir, 'test')

# # Tạo các thư mục con 'train', 'val' và 'test'
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Danh sách các nhãn
# labels = ['dog', 'cat']

# # Tỷ lệ train:val:test
# train_ratio = 0.7
# val_ratio = 0.2
# test_ratio = 0.1

# # Lặp qua các nhãn
# for label in labels:
#     # Tạo các thư mục con 'dog' và 'cat' trong 'train', 'val' và 'test'
#     os.makedirs(os.path.join(train_dir, label), exist_ok=True)
#     os.makedirs(os.path.join(val_dir, label), exist_ok=True)
#     os.makedirs(os.path.join(test_dir, label), exist_ok=True)

#     # Đường dẫn tới các tệp thuộc nhãn hiện tại
#     label_files = [file for file in os.listdir(file_dir) if file.startswith(label)]
#     print(label_files[0])
#     # Sắp xếp ngẫu nhiên các tệp
#     random.shuffle(label_files)

#     # Số lượng tệp trong từng thư mục con
#     num_files = len(label_files)
#     num_train = int(num_files * train_ratio)
#     num_val = int(num_files * val_ratio)
#     num_test = num_files - num_train - num_val

#     # Chia các tệp vào các thư mục con tương ứng
#     train_files = label_files[:num_train]
#     val_files = label_files[num_train:num_train + num_val]
#     test_files = label_files[num_train + num_val:]

#     for file in train_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(train_dir, label, file)
#         shutil.copy2(src, dst)

#     for file in val_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(val_dir, label, file)
#         shutil.copy2(src, dst)

#     for file in test_files:
#         src = os.path.join(file_dir, file)
#         dst = os.path.join(test_dir, label, file)
#         print(src)
#         print(dst)
#         shutil.copy2(src, dst)

## Cur

In [5]:
img_size = (224 , 224)
batch_size = 8
img_shape = (img_size[0] , img_size[1] , 3)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_directory(train_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

valid_gen = ts_gen.flow_from_directory(val_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

test_gen = ts_gen.flow_from_directory(test_dir , target_size = img_size , class_mode = 'binary' ,
                                       color_mode = 'rgb' , shuffle = False , batch_size = batch_size)

Found 3499 images belonging to 2 classes.
Found 999 images belonging to 2 classes.
Found 502 images belonging to 2 classes.


In [ ]:
# # Lấy các đường dẫn đến ảnh.
# # image_path = list(paths.list_images('data/dogs-vs-cats/train/train'))
# image_path = list(paths.list_images('/content/drive/MyDrive/5_AIADVANCE/GK_AI_Advance/dogs-vs-cats'))

# # Đổi vị trí ngẫu nhiên các đường dẫn ảnh
# random.shuffle(image_path)

# # Chỉ lấy 5000 dòng đầu tiên (5000 ảnh)
# # image_path = image_path[:5000]
# # print(image_path)

In [ ]:
# labels = [p.split(os.path.sep)[-1] for p in image_path]
# labels = [label.split('.')[-3] for label in labels]
# print(labels)
# # Chuyển tên các loài hoa thành số
# le = LabelEncoder()
# labels = le.fit_transform(labels)
# print(labels)

['dog', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'cat', 'dog', 'cat', 'dog', 'cat', 'dog', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'cat', 'cat', 'cat', 'cat'

In [ ]:
# # Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
# list_image = []
# for (j, imagePath) in enumerate(image_path):
#     # Load ảnh và resize về kích thước (224, 224)
#     image = load_img(imagePath, target_size=(224, 224))

#     # Chuyển ảnh thành mảng numpy
#     image = img_to_array(image)

#     # Mở rộng chiều của mảng numpy
#     image = np.expand_dims(image, 0)

#     # Tiền xử lý ảnh theo tiêu chuẩn của mô hình VGG16
#     image = imagenet_utils.preprocess_input(image)

#     # Thêm ảnh đã tiền xử lý vào danh sách
#     list_image.append(image)

# # Chuyển danh sách các ảnh thành mảng numpy một chiều
# list_image = np.vstack(list_image)

In [ ]:
# print(list_image.shape)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

#### Xây dựng Model

In [6]:
'''
ResNet18/34/50/101/152 in TensorFlow2.

Reference:
[1] He, Kaiming, et al.
    "Deep residual learning for image recognition."
    Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.
'''
import tensorflow as tf
from tensorflow.keras import layers
import sys

class BasicBlock(tf.keras.Model):
    expansion = 1

    def __init__(self, in_channels, out_channels, strides=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()

        if strides != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(self.expansion*out_channels, kernel_size=1, strides=strides, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = layers.add([self.shortcut(x), out])
        out = tf.keras.activations.relu(out)
        return out

class BottleNeck(tf.keras.Model):
    expansion = 4

    def __init__(self, in_channels, out_channels, strides=1):
        super(BottleNeck, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=1, use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv2D(self.expansion*out_channels, kernel_size=1, use_bias=False)
        self.bn3 = layers.BatchNormalization()

        if strides != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(self.expansion*out_channels, kernel_size=1, strides=strides, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = tf.keras.activations.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = layers.add([self.shortcut(x), out])
        out = tf.keras.activations.relu(out)
        return out

class BuildResNet(tf.keras.Model):
    def __init__(self, block, num_blocks, num_classes):
        super(BuildResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.layer1 = self._make_layer(block, 64, num_blocks[0], strides=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], strides=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], strides=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], strides=2)
        self.avg_pool2d = layers.AveragePooling2D(pool_size=4)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool2d(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

    def _make_layer(self, block, out_channels, num_blocks, strides):
        stride = [strides] + [1]*(num_blocks-1)
        layer = []
        for s in stride:
            layer += [block(self.in_channels, out_channels, s)]
            self.in_channels = out_channels * block.expansion
        return tf.keras.Sequential(layer)

def ResNet(model_type, num_classes):
    if model_type == 'resnet18':
        return BuildResNet(BasicBlock, [2, 2, 2, 2], num_classes)
    elif model_type == 'resnet34':
        return BuildResNet(BasicBlock, [3, 4, 6, 3], num_classes)
    elif model_type == 'resnet50':
        return BuildResNet(BottleNeck, [3, 4, 6, 3], num_classes)
    elif model_type == 'resnet101':
        return BuildResNet(BottleNeck, [3, 4, 23, 3], num_classes)
    elif model_type == 'resnet152':
        return BuildResNet(BottleNeck, [3, 8, 36, 3], num_classes)
    else:
        sys.exit(ValueError("{:s} is currently not supported.".format(model_type)))




In [7]:
resnet50 = ResNet(model_type = 'resnet50', num_classes = 2)

In [ ]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, Flatten, Dense
from keras.models import Model

def residual_block(input_tensor, filters, kernel_size=(3, 3), strides=(1, 1), activation='relu'):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Add()([x, input_tensor])
    x = Activation(activation)(x)
    return x

input_tensor = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_tensor)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Building Residual blocks
for _ in range(3):  # Example: 3 residual blocks
    x = residual_block(x, 32)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output)

In [8]:
resnet50.build((1, 224, 224, 3))
resnet50.summary()

Model: "build_res_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1728      
                                                                 
 batch_normalization (Batch  multiple                  256       
 Normalization)                                                  
                                                                 
 sequential_1 (Sequential)   (1, 224, 224, 256)        218624    
                                                                 
 sequential_3 (Sequential)   (1, 112, 112, 512)        1226752   
                                                                 
 sequential_5 (Sequential)   (1, 56, 56, 1024)         7118848   
                                                                 
 sequential_7 (Sequential)   (1, 28, 28, 2048)         14987264  
                                                     

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 111, 111, 32)         9248      ['max_pooling2d[0][0]']   

#### Fit model

In [ ]:
# Tạo đối tượng optimizer  # SGD
optimizer = tf.keras.optimizers.Adam()

# Thiết lập ModelCheckpoint
save_best_only = True
checkpoint = ModelCheckpoint('models/model-ResNet50-{epoch:03d}.keras',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=save_best_only,
                             mode='auto')

# Xây dựng quy trình huấn luyện
resnet50.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình với dữ liệu đã chuẩn bị
# H = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2, callbacks=[checkpoint])
H = resnet50.fit(
    train_gen,
    steps_per_epoch = train_gen.samples // train_gen.batch_size,
    validation_data = valid_gen,
    validation_steps = valid_gen.samples // valid_gen.batch_size,
    epochs = 20,
    callbacks=[checkpoint]
)

Epoch 1/20
 38/437 [=>............................] - ETA: 10:07:19 - loss: 15.0864 - accuracy: 0.5033

In [ ]:
# Vẽ đồ thị loss, accuracy của traning set và validation set
fig = plt.figure()
numOfEpoch = 10
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), H.history['accuracy'], label='accuracy') # sử dụng từ khóa accuracy thay vì acc
plt.plot(np.arange(0, numOfEpoch), H.history['val_accuracy'], label='validation accuracy') # sử dụng từ khóa val_accuracy thay vì val_acc
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()

In [ ]:
from keras.models import load_model
model = load_model('models/model-ResNet50-009.keras')

score = model.evaluate(X_test, y_test, verbose=1)
print(score)

In [ ]:
model.summary()

In [ ]:
# Đánh giá model
preds = model.predict(X_test) # DỰ ĐOÁN VÀ ĐÁNH GIÁ
preds = np.argmax(preds, axis=1)
print(classification_report(y_test, preds))

In [ ]:
# 10. Dự đoán ảnh
plt.imshow(X_test[0])
plt.axis('off')
plt.show()

y_predict = model.predict(np.expand_dims(X_test[0], axis=0))
print('Giá trị dự đoán: ', np.argmax(y_predict))

In [ ]:
# Tạo ma trận nhầm lẫn
cm = confusion_matrix(y_test, preds)

# Chuyển nhãn số thành tên
label_names = le.inverse_transform(np.unique(y_test))
label_names = [label.split('/')[-1] for label in label_names]
# Sắp xếp lại ma trận nhầm lẫn
sorted_cm = cm[np.argsort(label_names)][:, np.argsort(label_names)]

# Tạo dataframe từ ma trận nhầm lẫn đã sắp xếp lại
df_cm = pd.DataFrame(sorted_cm, index=label_names, columns=label_names)

# Vẽ ma trận nhầm lẫn
plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()
